Coucou

In [1]:
import numpy as np
import pandas as pd
import os
import math
from matplotlib import pyplot as plt
from model.track_collection import TrackCollection
from utils.collection_splitter import splitter

In [22]:
NUMBER_OF_USERS = 5

# loading track collection
track_collection = TrackCollection()
track_collection.load(os.path.join('data', 'track_collection_test.json'))
df_track_collection = track_collection.to_dataframe()
track_list = df_track_collection[['id']]
track_list

,id
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [23]:
# user_collections = splitter(track_collection, 5, 0.2)

# user_dfs = []
# for user_collection in user_collections:
#     user_dfs.append(user_collection.to_dataframe())
user_dfs = []
for i in range(NUMBER_OF_USERS):
    tc = TrackCollection()
    print("FILE", str(i+1)+'.json')
    tc.load(os.path.join('data', 'users', str(i+1)+'.json'))
    ndf = tc.to_dataframe()[['id', 'rating_score']]
    user_matrix = track_list.merge(ndf, on='id', how='left').fillna(0)
    user_dfs.append(user_matrix[['id', 'rating_score']])

user_dfs[3]

FILE 1.json
FILE 2.json
FILE 3.json
FILE 4.json
FILE 5.json


,id,rating_score
0,0,0.6
1,1,0.5
2,2,0
3,3,0
4,4,0.2
5,5,0.8
6,6,0.4
7,7,0
8,8,0
9,9,0


In [24]:
matrix = track_list
# matrix = matrix.merge(user_dfs[0], on='id', how='left').rename(columns = {'rating_score': 'user_0'})
for index, udf in enumerate(user_dfs):
    matrix = matrix.merge(user_dfs[index], on='id', how='left').rename(columns = {'rating_score': 'user_' + str(index)})
    

# ndf = user_dfs[0][['id', 'rating_score']]
# matrix.merge(ndf, on='id', how='left').rename(columns = {'rating_score': 'user_0'})
# i = 0
# for udf in user_dfs:
#     ndf = udf[['id', 'rating_score']]
#     matrix = matrix.merge(ndf, on='id', how='left').rename(columns = {'rating_score': 'user_'+str(i)})
#     i += 1

matrix = matrix.drop('id', axis=1)
matrix = matrix.filter(regex='^user_[0-9]+$').fillna(0)
# matrix = matrix.transpose()
matrix = matrix.astype('float')
matrix
# new

,user_0,user_1,user_2,user_3,user_4
0,0.5,0.00,0.2,0.6,0.9
1,0.4,0.60,0.1,0.5,0.0
2,0.0,0.00,0.0,0.0,0.0
3,0.0,0.00,0.0,0.0,0.4
4,0.0,0.20,0.0,0.2,0.0
5,0.0,0.65,0.0,0.8,0.0
6,0.0,0.80,0.0,0.4,0.0
7,0.4,0.20,0.0,0.0,0.0
8,0.2,0.00,0.3,0.0,0.0
9,0.8,0.30,0.9,0.0,0.0


In [25]:
def compute_normalize_matrix(df):
    for index, line in df.iterrows():
        sum = 0
        n = 0
        for el in line:
            if el > 0:
                sum += el
                n += 1
        if n != 0:
            line_mean = sum / n
        else:
            line_mean = 0

        for el_idx, el in enumerate(line):
            if el > 0:
                line[el_idx] = el - line_mean
#             else:
#                 line[el_idx] = math.nan
        df.loc[index] = line
    return df
        


def cos_measure(df, index_x, index_y):
    """Takes a dataframe representing the matrix and the line indexes of the elements to compare"""
    rx = df.iloc[index_x]
    ry = df.iloc[index_y]
    print("rx", rx)
    print("ry", ry)
    sc = np.dot(rx, ry)
    return sc / (np.linalg.norm(rx) * np.linalg.norm(ry))


def rating_mean(df, track_index):
    track = df.iloc[track_index]
    if np.count_nonzero(track) == 0:
        return 0
    return np.sum(track) / np.count_nonzero(track)
    

def pearson_measure(df, index_x, index_y):
   # comuting the center of each value
    mean_x = rating_mean(df, index_x)
    mean_y  = rating_mean(df, index_y)
    
    # we get the two elemnts we are interested in
    df_x = df.iloc[index_x:index_x + 1]
    df_y = df.iloc[index_y:index_y + 1]
    filtered_df = pd.concat([df_x, df_y])
    
    # we remove the columns containing empty measures (cannot be compared)
    filtered_df = filtered_df.loc[:, (~filtered_df.isin([0])).all(axis=0)]
#     print("FIL", filtered_df)

    upper_sum = 0
    for index, col in enumerate(filtered_df):
        upper_sum += (filtered_df.iat[0, index] - mean_x) * (filtered_df.iat[1, index] - mean_y)
    
    lower_sum_x = 0
    lower_sum_y = 0
    for index, col in enumerate(df):
        if df.iat[index_x, index] != 0:
            lower_sum_x += (df.iat[index_x, index] - mean_x) ** 2
        if df.iat[index_y, index] != 0:
            lower_sum_y += (df.iat[index_y, index] - mean_y) ** 2

#     print("LOW", lower_sum_x, lower_sum_y)
    if lower_sum_x == 0 or lower_sum_y == 0:
        return - math.inf
    return upper_sum / (math.sqrt(lower_sum_x) * math.sqrt(lower_sum_y))

In [26]:
n_matrix = compute_normalize_matrix(matrix)
print(cos_measure(n_matrix, 0, 1))
n_matrix

rx user_0   -0.05
user_1    0.00
user_2   -0.35
user_3    0.05
user_4    0.35
Name: 0, dtype: float64
ry user_0    0.0
user_1    0.2
user_2   -0.3
user_3    0.1
user_4    0.0
Name: 1, dtype: float64
0.587974732207


,user_0,user_1,user_2,user_3,user_4
0,-0.050000,0.000000,-0.350000,0.050,0.350
1,0.000000,0.200000,-0.300000,0.100,0.000
2,0.000000,0.000000,0.000000,0.000,0.000
3,0.000000,0.000000,0.000000,0.000,0.000
4,0.000000,0.000000,0.000000,0.000,0.000
5,0.000000,-0.075000,0.000000,0.075,0.000
6,0.000000,0.200000,0.000000,-0.200,0.000
7,0.100000,-0.100000,0.000000,0.000,0.000
8,-0.050000,0.000000,0.050000,0.000,0.000
9,0.133333,-0.366667,0.233333,0.000,0.000


In [27]:
height = matrix.shape[0]
sim_matrix = np.empty((height, height))
columns = matrix.index.values
for i in range(height): # for each track
    for j in range(height):
        sim_matrix[i][j] = pearson_measure(matrix, i, j)
sim_matrix = pd.DataFrame(sim_matrix, columns=matrix.index.values)
sim_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.000000,0.587975,-inf,-inf,-inf,0.070711,-0.070711,-0.070711,-0.424264,-0.388615,-inf,0.240887,-0.565685,-0.565685,-inf,0.494975,-0.535262,-0.083563,0.081650,0.141421
1,0.587975,1.000000,-inf,-inf,-inf,-0.188982,0.188982,-0.377964,-0.566947,-0.842651,-inf,-0.245256,0.000000,0.000000,-inf,-0.377964,-0.921038,0.322590,0.545545,0.188982
2,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
3,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
4,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
5,0.070711,-0.188982,-inf,-inf,-inf,1.000000,-1.000000,0.500000,0.000000,0.570323,-inf,-0.648886,0.000000,0.000000,-inf,0.500000,0.362933,-0.919145,0.577350,0.500000
6,-0.070711,0.188982,-inf,-inf,-inf,-1.000000,1.000000,-0.500000,0.000000,-0.570323,-inf,0.648886,0.000000,0.000000,-inf,-0.500000,-0.362933,0.919145,-0.577350,-0.500000
7,-0.070711,-0.377964,-inf,-inf,-inf,0.500000,-0.500000,1.000000,-0.500000,0.777714,-inf,-0.081111,0.500000,0.500000,-inf,0.500000,0.155543,-0.787839,0.000000,-0.500000
8,-0.424264,-0.566947,-inf,-inf,-inf,0.000000,0.000000,-0.500000,1.000000,0.155543,-inf,0.081111,-0.500000,-0.500000,-inf,0.000000,0.777714,0.196960,-0.288675,0.500000
9,-0.388615,-0.842651,-inf,-inf,-inf,0.570323,-0.570323,0.777714,0.155543,1.000000,-inf,-0.033643,0.207390,0.207390,-inf,0.570323,0.741935,-0.755679,-0.209540,-0.207390


In [28]:
height = n_matrix.shape[0]
n_sim_matrix = np.empty((height, height))
columns = n_matrix.index.values
for i in range(height): # for each track
    for j in range(height):
        n_sim_matrix[i][j] = cos_measure(n_matrix, i, j)
n_sim_matrix = pd.DataFrame(n_sim_matrix, columns=n_matrix.index.values)
n_sim_matrix

rx user_0   -0.05
user_1    0.00
user_2   -0.35
user_3    0.05
user_4    0.35
Name: 0, dtype: float64
ry user_0   -0.05
user_1    0.00
user_2   -0.35
user_3    0.05
user_4    0.35
Name: 0, dtype: float64
rx user_0   -0.05
user_1    0.00
user_2   -0.35
user_3    0.05
user_4    0.35
Name: 0, dtype: float64
ry user_0    0.0
user_1    0.2
user_2   -0.3
user_3    0.1
user_4    0.0
Name: 1, dtype: float64
rx user_0   -0.05
user_1    0.00
user_2   -0.35
user_3    0.05
user_4    0.35
Name: 0, dtype: float64
ry user_0    0.0
user_1    0.0
user_2    0.0
user_3    0.0
user_4    0.0
Name: 2, dtype: float64
rx user_0   -0.05
user_1    0.00
user_2   -0.35
user_3    0.05
user_4    0.35
Name: 0, dtype: float64
ry user_0    0.0
user_1    0.0
user_2    0.0
user_3    0.0
user_4    0.0
Name: 3, dtype: float64
rx user_0   -0.05
user_1    0.00
user_2   -0.35
user_3    0.05
user_4    0.35
Name: 0, dtype: float64
ry user_0    0.0
user_1    0.0
user_2    0.0
user_3    0.0
user_4    0.0
Name: 4, dtype: float64


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars



rx user_0    0.0
user_1    0.2
user_2    0.0
user_3   -0.2
user_4    0.0
Name: 6, dtype: float64
ry user_0    0.000
user_1    0.025
user_2    0.025
user_3   -0.175
user_4    0.125
Name: 11, dtype: float64
rx user_0    0.0
user_1    0.2
user_2    0.0
user_3   -0.2
user_4    0.0
Name: 6, dtype: float64
ry user_0    0.15
user_1    0.00
user_2    0.00
user_3    0.00
user_4   -0.15
Name: 12, dtype: float64
rx user_0    0.0
user_1    0.2
user_2    0.0
user_3   -0.2
user_4    0.0
Name: 6, dtype: float64
ry user_0    0.15
user_1    0.00
user_2    0.00
user_3    0.00
user_4   -0.15
Name: 13, dtype: float64
rx user_0    0.0
user_1    0.2
user_2    0.0
user_3   -0.2
user_4    0.0
Name: 6, dtype: float64
ry user_0    0.0
user_1    0.0
user_2    0.0
user_3    0.0
user_4    0.0
Name: 14, dtype: float64
rx user_0    0.0
user_1    0.2
user_2    0.0
user_3   -0.2
user_4    0.0
Name: 6, dtype: float64
ry user_0    0.00
user_1   -0.05
user_2    0.00
user_3    0.00
user_4    0.05
Name: 15, dtype: float64

Name: 12, dtype: float64
ry user_0    0.0
user_1    0.0
user_2    0.0
user_3    0.0
user_4    0.0
Name: 14, dtype: float64
rx user_0    0.15
user_1    0.00
user_2    0.00
user_3    0.00
user_4   -0.15
Name: 12, dtype: float64
ry user_0    0.00
user_1   -0.05
user_2    0.00
user_3    0.00
user_4    0.05
Name: 15, dtype: float64
rx user_0    0.15
user_1    0.00
user_2    0.00
user_3    0.00
user_4   -0.15
Name: 12, dtype: float64
ry user_0   -0.133333
user_1   -0.233333
user_2    0.366667
user_3    0.000000
user_4    0.000000
Name: 16, dtype: float64
rx user_0    0.15
user_1    0.00
user_2    0.00
user_3    0.00
user_4   -0.15
Name: 12, dtype: float64
ry user_0   -0.15
user_1    0.45
user_2    0.00
user_3   -0.25
user_4   -0.05
Name: 17, dtype: float64
rx user_0    0.15
user_1    0.00
user_2    0.00
user_3    0.00
user_4   -0.15
Name: 12, dtype: float64
ry user_0    0.0
user_1    0.0
user_2   -0.1
user_3    0.2
user_4   -0.1
Name: 18, dtype: float64
rx user_0    0.15
user_1    0.00
user_

Name: 17, dtype: float64
rx user_0    0.0
user_1    0.0
user_2   -0.1
user_3    0.2
user_4   -0.1
Name: 18, dtype: float64
ry user_0    0.0
user_1    0.0
user_2   -0.1
user_3    0.2
user_4   -0.1
Name: 18, dtype: float64
rx user_0    0.0
user_1    0.0
user_2   -0.1
user_3    0.2
user_4   -0.1
Name: 18, dtype: float64
ry user_0   -0.4
user_1    0.0
user_2    0.0
user_3    0.4
user_4    0.0
Name: 19, dtype: float64
rx user_0   -0.4
user_1    0.0
user_2    0.0
user_3    0.4
user_4    0.0
Name: 19, dtype: float64
ry user_0   -0.05
user_1    0.00
user_2   -0.35
user_3    0.05
user_4    0.35
Name: 0, dtype: float64
rx user_0   -0.4
user_1    0.0
user_2    0.0
user_3    0.4
user_4    0.0
Name: 19, dtype: float64
ry user_0    0.0
user_1    0.2
user_2   -0.3
user_3    0.1
user_4    0.0
Name: 1, dtype: float64
rx user_0   -0.4
user_1    0.0
user_2    0.0
user_3    0.4
user_4    0.0
Name: 19, dtype: float64
ry user_0    0.0
user_1    0.0
user_2    0.0
user_3    0.0
user_4    0.0
Name: 2, dtype: f

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.000000,0.587975,NaN,NaN,NaN,0.070711,-0.070711,-0.070711,-0.424264,-0.388615,NaN,0.240887,-0.565685,-0.565685,NaN,0.494975,-0.535262,-0.083563,0.081650,0.141421
1,0.587975,1.000000,NaN,NaN,NaN,-0.188982,0.188982,-0.377964,-0.566947,-0.842651,NaN,-0.245256,0.000000,0.000000,NaN,-0.377964,-0.921038,0.322590,0.545545,0.188982
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.070711,-0.188982,NaN,NaN,NaN,1.000000,-1.000000,0.500000,0.000000,0.570323,NaN,-0.648886,0.000000,0.000000,NaN,0.500000,0.362933,-0.919145,0.577350,0.500000
6,-0.070711,0.188982,NaN,NaN,NaN,-1.000000,1.000000,-0.500000,0.000000,-0.570323,NaN,0.648886,0.000000,0.000000,NaN,-0.500000,-0.362933,0.919145,-0.577350,-0.500000
7,-0.070711,-0.377964,NaN,NaN,NaN,0.500000,-0.500000,1.000000,-0.500000,0.777714,NaN,-0.081111,0.500000,0.500000,NaN,0.500000,0.155543,-0.787839,0.000000,-0.500000
8,-0.424264,-0.566947,NaN,NaN,NaN,0.000000,0.000000,-0.500000,1.000000,0.155543,NaN,0.081111,-0.500000,-0.500000,NaN,0.000000,0.777714,0.196960,-0.288675,0.500000
9,-0.388615,-0.842651,NaN,NaN,NaN,0.570323,-0.570323,0.777714,0.155543,1.000000,NaN,-0.033643,0.207390,0.207390,NaN,0.570323,0.741935,-0.755679,-0.209540,-0.207390


In [17]:
# Predict the weighted average
def predict(df, sim_matrix, element_index, user_index):
    user_col = df.iloc[:, user_index]

    # compute predicted rating
    upper_sum = 0
    lower_sum = 0
    for index, el in enumerate(sim_matrix.iloc[:, element_index]):
        if el >= 0 and el <= 0.99:
            upper_sum += el * user_col[index]
            lower_sum += el
    if lower_sum == 0:
        return - math.inf
    return upper_sum / lower_sum


# we compute the predictions
userToPredict = []
userToPredict.append([2,3,4,5,6,10,11,14,15,18])
userToPredict.append([0,2,3,8,12,13,14,18,19])
userToPredict.append([2,3,4,5,6,7,10,12,13,14,15,17,19])
userToPredict.append([2,3,7,8,9,10,12,13,14,15,16])
userToPredict.append([1,2,4,5,6,7,8,9,10])
print(userToPredict)

print("PEARSON")
for i in range(5):
    for j in range(20):
        if predict(matrix, sim_matrix, j, i) > rating_mean(matrix, j) and j in userToPredict[i]:
            print("user_{} should listen music {}".format(i, j))

print("COS")
for i in range(5):
    for j in range(20):
        if predict(n_matrix, n_sim_matrix, j, i) > rating_mean(n_matrix, j) and j in userToPredict[i]:
            print("user_{} should listen music {}".format(i, j))

[[2, 3, 4, 5, 6, 10, 11, 14, 15, 18], [0, 2, 3, 8, 12, 13, 14, 18, 19], [2, 3, 4, 5, 6, 7, 10, 12, 13, 14, 15, 17, 19], [2, 3, 7, 8, 9, 10, 12, 13, 14, 15, 16], [1, 2, 4, 5, 6, 7, 8, 9, 10]]
PEARSON
COS
